### workbook using sql with pandas to analyze an activity log

### we want to analyze an activity log that contains a timestamp for each time the activity takes place (in this case, smoking). we will want to know the number of unique days the activity took place for each subject (user), and the difference between the number of times it happened on the first day vs the last day for each subject

In [7]:
import pandas as pd

In [8]:
from pandasql import sqldf

In [9]:
pysqldf = lambda q: sqldf(q, globals())

In [10]:
smoking_log_df = pd.read_csv("smoking_log.csv")

In [11]:
# data (fabricated, not real) showing user id and logged time for each activity as a timestamp
smoking_log_df

,id,user_id,time_stamp
0,1,1,6/7/14 12:38AM
1,2,1,6/8/14 12:39AM
2,3,1,6/8/14 12:40AM
3,4,1,6/9/14 12:38AM
4,5,2,6/7/14 12:38AM
5,6,2,6/8/14 12:39AM
6,7,2,6/8/14 12:40AM
7,8,2,6/9/14 12:38AM
8,9,2,6/7/14 12:38AM
9,10,3,6/8/14 12:39AM


In [13]:
# we only want dates.  Note we use sqlite syntax with pandasql
df_dates = pysqldf("SELECT id, user_id, SUBSTR(time_stamp, 0, 7) AS date FROM smoking_log_df")

In [14]:
df_dates

,id,user_id,date
0,1,1,6/7/14
1,2,1,6/8/14
2,3,1,6/8/14
3,4,1,6/9/14
4,5,2,6/7/14
5,6,2,6/8/14
6,7,2,6/8/14
7,8,2,6/9/14
8,9,2,6/7/14
9,10,3,6/8/14


In [15]:
# log the activity count per day
num_cig_per_day = pysqldf("SELECT user_id, date, count(user_id) as num_cig \
    FROM df_dates GROUP BY user_id, date")

In [16]:
num_cig_per_day

,user_id,date,num_cig
0,1,6/7/14,1
1,1,6/8/14,2
2,1,6/9/14,1
3,2,6/7/14,2
4,2,6/8/14,2
5,2,6/9/14,1
6,3,6/7/14,1
7,3,6/8/14,2
8,3,6/9/14,1
9,4,6/8/14,2


In [17]:
# query the unique days the activity took place, per user
uniq_user_days = pysqldf("SELECT DISTINCT user_id, date FROM df_dates")

In [18]:
uniq_user_days

,user_id,date
0,1,6/7/14
1,1,6/8/14
2,1,6/9/14
3,2,6/7/14
4,2,6/8/14
5,2,6/9/14
6,3,6/8/14
7,3,6/9/14
8,3,6/7/14
9,4,6/8/14


In [19]:
# how many unique days did the activity take place for each user 
count_uniq_days = pysqldf("SELECT user_id, COUNT(date) FROM uniq_user_days GROUP BY user_id")

In [20]:
count_uniq_days

,user_id,COUNT(date)
0,1,3
1,2,3
2,3,3
3,4,2


In [27]:
# get the first and last day the activity occurred for each user
min_dates = pysqldf("select user_id, min(date) AS min_date \
    FROM num_cig_per_day group by user_id")

In [28]:
max_dates = pysqldf("select user_id, max(date) AS max_date \
    FROM num_cig_per_day group by user_id")

In [29]:
min_dates

,user_id,min_date
0,1,6/7/14
1,2,6/7/14
2,3,6/7/14
3,4,6/8/14


In [30]:
max_dates

,user_id,max_date
0,1,6/9/14
1,2,6/9/14
2,3,6/9/14
3,4,6/9/14


In [ ]:
# TODO: what is the difference between the number of times the activity took place on the first
# day it happened and the last day it occurred?
# we didn't have time to do this, if you want to try it, please send in a pull request!